<a href="https://colab.research.google.com/github/kchamarty/pythonproject/blob/master/kumarchamarty_pythonv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Importing**


---

We have two data sets one containing the patient doctor visit information and the other containing the status of the patient active or not